# Projeto 2 - CDados

### Autores: Diego Guimarães Nuñez, Felipe Bakowski Nantes de Souza, Gustavo de Lima Mota

### Tema: Conseguimos prever o número de homicídios da cidade de São Paulo por ano?

### Features: PIB de São Paulo, Índice do Custo de Vida (ICV), Escolaridade, Tráfico de Drogas e Desemprego.

In [4]:
import basedosdados as bd
import pandas as pd

### Baixando as bases de dados utilizadas no projeto:

In [26]:
# Baixando dados desemprego #utilizando a base que foi alterada(havia um erro de digitação no original)!
desemprego = pd.read_excel("desemprego_alterado.xlsx")

In [27]:
desemprego 
# base de dados de desemprego
# nesse df observa-se a taxa de desocupação (%) da população brasileira ao longo dos anos (2012-2022)

,Taxa de desocupação,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 131,Unnamed: 132,Unnamed: 133,Unnamed: 134,Unnamed: 135,Unnamed: 136,Unnamed: 137,Unnamed: 138,Unnamed: 139,Unnamed: 140
0,NaN,p,jan-fev-mar 2012,fev-mar-abr 2012,mar-abr-mai 2012,abr-mai-jun 2012,mai-jun-jul 2012,jun-jul-ago 2012,jul-ago-set 2012,ago-set-out 2012,...,out-nov-dez 2022,nov-dez-jan 2023,dez-jan-fev 2023,jan-fev-mar 2023,fev-mar-abr 2023,mar-abr-mai 2023,abr-mai-jun 2023,mai-jun-jul 2023,jun-jul-ago 2023,jul-ago-set 2023
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Brasil,NaN,"8,0","7,8","7,7","7,6","7,5","7,4","7,1","7,0",...,"7,9","8,4","8,6","8,8","8,5","8,3","8,0","7,9","7,8","7,7"
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"Fonte: ""IBGE - Pesquisa Nacional por Amostra d...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
inep = bd.read_table(dataset_id='br_inep_saeb',
table_id='municipio',
billing_project_id="cdadosp2-404421")

Downloading: 100%|██████████| 1726300/1726300 [06:38<00:00, 4327.92rows/s]


In [8]:
inep # base de dados de níveis de escolaridade

,ano,rede,localizacao,sigla_uf,id_municipio,disciplina,serie,media,nivel_0,nivel_1,nivel_2,nivel_3,nivel_4,nivel_5,nivel_6,nivel_7,nivel_8,nivel_9,nivel_10
0,2011,Total - Estadual e Municipal,Rural,MG,3162450,MT,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2011,Total - Estadual e Municipal,Rural,RS,4302055,LP,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2011,"Total - Federal, Estadual e Municipal",Rural,MT,5103858,LP,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2011,Estadual,Total,MT,5107354,MT,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2011,"Total - Federal, Estadual e Municipal",Rural,MT,5101852,LP,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1726295,2021,"Total - Federal, Estadual e Municipal",Rural,SC,4219705,MT,5,212.93,0.00,3.83,26.96,6.19,22.23,20.77,6.19,10.75,3.10,0.00,0.0
1726296,2021,"Total - Federal, Estadual e Municipal",Total,SE,2802700,MT,9,231.05,24.25,16.99,27.96,16.73,8.72,3.03,1.31,0.00,1.01,0.00,NaN
1726297,2021,"Total - Federal, Estadual, Municipal e Privada",Total,PA,1500958,MT,5,176.34,5.18,20.34,27.96,23.64,12.94,6.72,1.34,1.56,0.00,0.32,0.0
1726298,2021,"Total - Federal, Estadual, Municipal e Privada",Total,PR,4115853,MT,5,241.99,1.44,0.00,1.31,12.21,14.82,28.02,21.83,10.64,9.74,0.00,0.0


In [9]:
pib = bd.read_table(dataset_id='br_ibge_pib',
table_id='municipio',
billing_project_id="cdadosp2-404421")

Downloading: 100%|██████████| 105830/105830 [00:17<00:00, 5975.02rows/s]


In [10]:
pib # base de dados dos pib´s por município do Brasil por ano

,id_municipio,ano,pib,impostos_liquidos,va,va_agropecuaria,va_industria,va_servicos,va_adespss
0,1100015,2002,111291000,7549000,103742000,27013000,9377000,24651000,42701000
1,1100023,2002,449593000,42666000,406927000,28590000,99923000,161852000,116562000
2,1100031,2002,31768000,1868000,29900000,10884000,1962000,5129000,11925000
3,1100049,2002,474443000,49623000,424820000,32810000,98293000,175749000,117968000
4,1100056,2002,79174000,8201000,70973000,11284000,6651000,25306000,27731000
...,...,...,...,...,...,...,...,...,...
105825,5222005,2020,587313000,50678000,536634000,237994000,31362000,198625000,68654000
105826,5222054,2020,418557000,33375000,385182000,158818000,75909000,102478000,47977000
105827,5222203,2020,121887000,7569000,114318000,28358000,22837000,29547000,33577000
105828,5222302,2020,218473000,8738000,209736000,115114000,26940000,34812000,32870000


In [11]:
icv = bd.read_table(dataset_id='br_sp_saopaulo_dieese_icv',
table_id='ano',
billing_project_id="cdadosp2-404421")

Downloading: 100%|██████████| 58/58 [00:00<00:00, 188.31rows/s]


In [12]:
icv # base de dados sobre a feature de variação anual do índice de custo de vida.

,ano,variacao_anual
0,1960,33.013597
1,1961,42.004960
2,1962,62.652008
3,1963,90.247638
4,1964,73.131252
5,1965,53.873224
6,1966,52.404392
7,1967,25.773380
8,1968,26.061802
9,1969,22.249454


In [13]:
homicidio = bd.read_table(dataset_id='br_sp_gov_ssp',
table_id='ocorrencias_registradas',
billing_project_id="cdadosp2-404421")

Downloading: 100%|██████████| 153876/153876 [00:28<00:00, 5458.03rows/s]


In [14]:
homicidio

,ano,mes,id_municipio,regiao_ssp,homicidio_doloso,numero_de_vitimas_em_homicidio_doloso,homicidio_doloso_por_acidente_de_transito,numero_de_vitimas_em_homicidio_doloso_por_acidente_de_transito,homicidio_culposo_por_acidente_de_transito,homicidio_culposo_outros,...,total_de_estupro,estupro,estupro_de_vulneravel,total_de_roubo_outros,roubo_outros,roubo_de_veiculo,roubo_a_banco,roubo_de_carga,furto_outros,furto_de_veiculo
0,2002,1,3500105,Presidente Prudente,0,0,0,0,0,0,...,0,<NA>,<NA>,0,<NA>,0,0,0,21,0
1,2002,2,3500105,Presidente Prudente,0,0,0,0,0,0,...,0,<NA>,<NA>,0,<NA>,0,0,0,32,0
2,2002,3,3500105,Presidente Prudente,0,0,0,0,0,0,...,0,<NA>,<NA>,0,<NA>,0,0,0,36,0
3,2002,4,3500105,Presidente Prudente,0,0,0,0,0,0,...,0,<NA>,<NA>,1,<NA>,0,0,0,45,0
4,2002,5,3500105,Presidente Prudente,0,0,0,0,0,0,...,0,<NA>,<NA>,1,<NA>,0,0,0,35,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153871,2014,8,3500204,São José do Rio Preto,0,0,0,0,0,0,...,1,<NA>,<NA>,0,<NA>,0,0,0,3,0
153872,2014,9,3500204,São José do Rio Preto,0,0,0,0,0,0,...,0,<NA>,<NA>,0,<NA>,0,0,0,0,0
153873,2014,10,3500204,São José do Rio Preto,0,0,0,0,0,0,...,0,<NA>,<NA>,0,<NA>,0,0,0,3,0
153874,2014,11,3500204,São José do Rio Preto,0,0,0,0,0,0,...,0,<NA>,<NA>,0,<NA>,0,0,0,4,1


In [15]:
ivs = bd.read_table(dataset_id='br_ipea_avs',
table_id='municipio',
billing_project_id="cdadosp2-404421")

Downloading: 100%|██████████| 319681/319681 [06:29<00:00, 821.14rows/s] 


In [16]:
ivs # colunas que serão usadas para feature: "ivs", "ivs_infraestrutura_urbana", "ivs_capital_humano" e "ivs_renda_trabalho"
# índice de vulnerabilidade social.

,ano,sigla_uf,id_municipio,raca_cor,sexo,localizacao,ivs,ivs_infraestrutura_urbana,ivs_capital_humano,ivs_renda_trabalho,...,populacao_15_mais,populacao_15_17,populacao_15_24,populacao_16_18,populacao_18_mais,populacao_18_20,populacao_18_24,populacao_19_21,populacao_25_mais,populacao_65_mais
0,2000,AP,1600105,branco,total,total,0.572,0.775,0.573,0.369,...,606,80,225,76,526,56,145,58,381,44
1,2000,AP,1600279,branco,total,total,0.677,0.880,0.649,0.503,...,4765,507,1855,509,4258,761,1347,717,2911,129
2,2000,AP,1600303,branco,total,total,0.379,0.292,0.435,0.411,...,52086,5786,18687,5953,46301,5854,12902,5839,33399,2392
3,2000,AP,1600402,branco,total,total,0.656,0.777,0.622,0.569,...,889,105,335,120,784,120,229,104,555,55
4,2000,AP,1600535,branco,total,total,0.615,0.713,0.691,0.441,...,1387,101,452,117,1285,176,351,174,935,92
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
319676,2010,MG,3104007,negro,mulher,rural,NaN,NaN,NaN,NaN,...,258,0,98,0,258,30,98,41,160,8
319677,2010,MG,3115508,negro,mulher,rural,NaN,NaN,NaN,NaN,...,77,0,14,0,77,0,14,0,63,0
319678,2010,MG,3133105,negro,mulher,rural,NaN,NaN,NaN,NaN,...,121,0,5,0,121,5,5,5,116,19
319679,2010,MG,3168705,negro,mulher,rural,NaN,NaN,NaN,NaN,...,38,0,0,0,38,0,0,0,38,0


## Para esse projeto decidimos fazer uma modelagem do número de homicidios em SP e tentar prever como esse evento atua em função de 5 features distintas. As quais são : Desemprego, nível de escolaridade, PIB, custo de vida (ICV) e o índice de vulnerabilidade social (IVC)

### A meta é conseguir criar um modelo satisfatório para e validá-lo com dados reais, a fim de, termos uma ideia melhor de como a criminalidade cresce no Brasil e tomarmos medidas para nos proteger

#### Primeiramente, vamos tratar a base de dados de homicidios e deixá-la mais visualizável

In [17]:
homicidio_colunas = homicidio.loc[:,('ano', 'mes', 'regiao_ssp', 'homicidio_doloso', 'homicidio_culposo_outros')] #selecionando colunas
homicidio['Homicidio_total'] = homicidio.loc[:, ['homicidio_doloso', 'homicidio_culposo_outros']].sum(axis=1) #homicidio total
homicidio_filtrado = homicidio.loc[:,('ano', 'mes', 'regiao_ssp', 'Homicidio_total')] #selecionando colunas

In [18]:
homicidio_filtrado

,ano,mes,regiao_ssp,Homicidio_total
0,2002,1,Presidente Prudente,0.0
1,2002,2,Presidente Prudente,0.0
2,2002,3,Presidente Prudente,0.0
3,2002,4,Presidente Prudente,0.0
4,2002,5,Presidente Prudente,0.0
...,...,...,...,...
153871,2014,8,São José do Rio Preto,0.0
153872,2014,9,São José do Rio Preto,0.0
153873,2014,10,São José do Rio Preto,0.0
153874,2014,11,São José do Rio Preto,0.0


### Agora, devemos tratar a base de dados Desemprego e analisar sua correlação com a target

In [28]:
datas = desemprego.iloc[0] #meses/ano
valores = desemprego.iloc[2] #valores limpan

In [29]:
dados = {}
for mes in range(len(datas)):
    if mes > 1:
        dados[datas[mes]] = valores[mes]
desemprego_tratado = (pd.DataFrame(dados, index=['%']))

In [30]:
desemprego_tratado

,jan-fev-mar 2012,fev-mar-abr 2012,mar-abr-mai 2012,abr-mai-jun 2012,mai-jun-jul 2012,jun-jul-ago 2012,jul-ago-set 2012,ago-set-out 2012,set-out-nov 2012,out-nov-dez 2012,...,out-nov-dez 2022,nov-dez-jan 2023,dez-jan-fev 2023,jan-fev-mar 2023,fev-mar-abr 2023,mar-abr-mai 2023,abr-mai-jun 2023,mai-jun-jul 2023,jun-jul-ago 2023,jul-ago-set 2023
%,"8,0","7,8","7,7","7,6","7,5","7,4","7,1","7,0","6,8","6,9",...,"7,9","8,4","8,6","8,8","8,5","8,3","8,0","7,9","7,8","7,7"


### Agora que a base demprego está tratada, encontramos o problema de que a % de desemprego, na linha, é referente aos 3 meses analisados na coluna. Então, precisamos fazer a análise com respeito ao período estabelecido

### Vale notar:

### Iremos analisar dos anos 2012 até 2014

In [171]:
homicidio_2012 = homicidio_filtrado.loc[homicidio['ano'].isin([2012,2013,2014])]
homicidio_2012

,ano,mes,regiao_ssp,Homicidio_total
46200,2013,1,Presidente Prudente,0.0
46201,2013,2,Presidente Prudente,0.0
46202,2013,3,Presidente Prudente,0.0
46203,2013,4,Presidente Prudente,0.0
46204,2013,5,Presidente Prudente,0.0
...,...,...,...,...
153871,2014,8,São José do Rio Preto,0.0
153872,2014,9,São José do Rio Preto,0.0
153873,2014,10,São José do Rio Preto,0.0
153874,2014,11,São José do Rio Preto,0.0


## Agora necessitamos filtrar a base desemprego para ter apenas os anos de interseção com a base homicídio

In [172]:
dados = {}
for k,v in desemprego_tratado.items():
    if '2012' in k or '2013' in k or '2014' in k:
        dados[k] = v
desemprego_2012 = pd.DataFrame(dados, index=['%'])
desemprego_2012

,jan-fev-mar 2012,fev-mar-abr 2012,mar-abr-mai 2012,abr-mai-jun 2012,mai-jun-jul 2012,jun-jul-ago 2012,jul-ago-set 2012,ago-set-out 2012,set-out-nov 2012,out-nov-dez 2012,...,jan-fev-mar 2014,fev-mar-abr 2014,mar-abr-mai 2014,abr-mai-jun 2014,mai-jun-jul 2014,jun-jul-ago 2014,jul-ago-set 2014,ago-set-out 2014,set-out-nov 2014,out-nov-dez 2014
%,"8,0","7,8","7,7","7,6","7,5","7,4","7,1","7,0","6,8","6,9",...,"7,2","7,2","7,1","6,9","7,0","7,0","6,9","6,7","6,6","6,6"


## Então, criaremos um df com as features e target como colunas e as datas como indices de linhas, para que, consigamos realizar uma análise quantitativa 

In [173]:
desemprego_2012_x = desemprego_2012.transpose().reset_index()
for header,columns in desemprego_2012_x.items():
    if header == 'index':
        j = 0
        for i in columns:
            mes = i[:3]
            ano = i[-4:]
            if mes == 'jan':
                mes = 1
                desemprego_2012_x['index'].iloc[j] = f'{mes}-{ano}'
            elif mes == 'fev':
                mes = 2
                desemprego_2012_x['index'].iloc[j] = f'{mes}-{ano}'
            elif mes == 'mar':
                mes = 3
                desemprego_2012_x['index'].iloc[j] = f'{mes}-{ano}'
            elif mes == 'abr':
                mes = 4
                desemprego_2012_x['index'].iloc[j] = f'{mes}-{ano}'
            elif mes == 'mai':
                mes = 5
                desemprego_2012_x['index'].iloc[j] = f'{mes}-{ano}'
            elif mes == 'jun':
                mes = 6
                desemprego_2012_x['index'].iloc[j] = f'{mes}-{ano}'
            elif mes == 'jul':
                mes = 7
                desemprego_2012_x['index'].iloc[j] = f'{mes}-{ano}'
            elif mes == 'ago':
                mes = 8
                desemprego_2012_x['index'].iloc[j] = f'{mes}-{ano}'
            elif mes == 'set':
                mes = 9
                desemprego_2012_x['index'].iloc[j] = f'{mes}-{ano}'
            elif mes == 'out':
                mes = 10
                desemprego_2012_x['index'].iloc[j] = f'{mes}-{ano}'
            elif mes == 'nov':
                mes = 11
                desemprego_2012_x['index'].iloc[j] = f'{mes}-{ano}'
            elif mes == 'dez':
                mes = 12
                desemprego_2012_x['index'].iloc[j] = f'{mes}-{ano}'
            j += 1

In [174]:
desemprego_2012_x

,index,%
0,1-2012,"8,0"
1,2-2012,"7,8"
2,3-2012,"7,7"
3,4-2012,"7,6"
4,5-2012,"7,5"
5,6-2012,"7,4"
6,7-2012,"7,1"
7,8-2012,"7,0"
8,9-2012,"6,8"
9,10-2012,"6,9"


In [175]:
anos = list(homicidio_2012.ano)

In [176]:
meses = list(homicidio_2012.mes)

In [177]:
homicidio_2012_x = homicidio_2012.reset_index()

In [178]:
for indice in range(len(anos)):
    homicidio_2012_x['index'].iloc[indice] = f'{meses[indice]}-{anos[indice]}'

In [179]:
homicidio_2012_x

,index,ano,mes,regiao_ssp,Homicidio_total
0,1-2013,2013,1,Presidente Prudente,0.0
1,2-2013,2013,2,Presidente Prudente,0.0
2,3-2013,2013,3,Presidente Prudente,0.0
3,4-2013,2013,4,Presidente Prudente,0.0
4,5-2013,2013,5,Presidente Prudente,0.0
...,...,...,...,...,...
23023,8-2014,2014,8,São José do Rio Preto,0.0
23024,9-2014,2014,9,São José do Rio Preto,0.0
23025,10-2014,2014,10,São José do Rio Preto,0.0
23026,11-2014,2014,11,São José do Rio Preto,0.0


In [180]:
#homicidio_2012_x = homicidio_2012_x[['index','regiao_ssp','Homicidio_total']].dropna()

In [181]:
homicidio_2012_x

,index,ano,mes,regiao_ssp,Homicidio_total
0,1-2013,2013,1,Presidente Prudente,0.0
1,2-2013,2013,2,Presidente Prudente,0.0
2,3-2013,2013,3,Presidente Prudente,0.0
3,4-2013,2013,4,Presidente Prudente,0.0
4,5-2013,2013,5,Presidente Prudente,0.0
...,...,...,...,...,...
23023,8-2014,2014,8,São José do Rio Preto,0.0
23024,9-2014,2014,9,São José do Rio Preto,0.0
23025,10-2014,2014,10,São José do Rio Preto,0.0
23026,11-2014,2014,11,São José do Rio Preto,0.0


In [182]:
df = desemprego_2012_x.join(homicidio_2012_x, how='inner', lsuffix='_datas')

In [184]:
df = df[['index_datas','%','regiao_ssp','Homicidio_total']].dropna()

In [185]:
df

,index_datas,%,regiao_ssp,Homicidio_total
0,1-2012,"8,0",Presidente Prudente,0.0
1,2-2012,"7,8",Presidente Prudente,0.0
2,3-2012,"7,7",Presidente Prudente,0.0
3,4-2012,"7,6",Presidente Prudente,0.0
4,5-2012,"7,5",Presidente Prudente,0.0
5,6-2012,"7,4",Presidente Prudente,0.0
6,7-2012,"7,1",Presidente Prudente,0.0
7,8-2012,"7,0",Presidente Prudente,0.0
8,9-2012,"6,8",Presidente Prudente,0.0
9,10-2012,"6,9",Presidente Prudente,0.0
